# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## 1. Compute the camera calibration using chessboard images

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip
from IPython.display import HTML

In [ ]:
def load_and_cal_chess_points(file_path):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    correct_imgs = []

    # Make a list of calibration images
    images = glob.glob(file_path + '/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)

        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            correct_imgs.append(img)
    return correct_imgs, objpoints, imgpoints

## 2. Undistort image with chessboard points 

In [ ]:
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [ ]:
correct_imgs, objpoints, imgpoints = load_and_cal_chess_points('./camera_cal/')

In [ ]:
for img in correct_imgs:
    undistorted = cal_undistort(img, objpoints, imgpoints)
    
    plt.figure()
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undistorted)
    ax2.set_title('Undistorted Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## 3. Transform image to HLS color space and use x gradient to create a thresholded binary image

In [ ]:
def color_and_gradient_threshold(img, s_thresh=(90, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return color_binary, combined_binary

## 4. Perspective transform image to birdeye view

In [ ]:
def warp(img):
    img_size = (img.shape[1], img.shape[0])
    src = np.float32([
        [685,450],
        [1100,720],
        [200,720],
        [595,450]
    ])
    dst = np.float32([
        [950,0],
        [950,720],
        [350,720],
        [350,0]
    ])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, Minv

## 5. Find lanes with histogram and sliding window

In [ ]:
def hist_and_slide_window(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    return out_img, leftx, lefty, rightx, righty


## 6. Measure curve rad in pixel space and real world space

In [ ]:
def fit_curve(warpped_img, leftx, lefty, rightx, righty):
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    ploty = np.linspace(0, warpped_img.shape[0]-1, warpped_img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    return left_fit, right_fit, left_fitx, right_fitx, ploty


def plot_curve(out_img, leftx, lefty, rightx, righty, left_fitx, right_fitx, ploty):
    plt.figure()
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.title("Curve", fontsize=30)


def cal_curverad(left_fit, right_fit, leftx, lefty, rightx, righty, ploty):
    result = []
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    result.append("Pixel curverad left %.2f right %.2f" % (left_curverad, right_curverad))
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30.0/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/600 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    result.append("Meter curverad left %.2fm right %.2fm" % (left_curverad, right_curverad))
    # Example values: 632.1 m    626.2 m
    
    center_xm = xm_per_pix * 1280 / 2
    left_xm = left_fit_cr[0]*((y_eval*ym_per_pix)**2) + left_fit_cr[1]*y_eval*ym_per_pix + left_fit_cr[2]
    right_xm = right_fit_cr[0]*((y_eval*ym_per_pix)**2) + right_fit_cr[1]*y_eval*ym_per_pix + right_fit_cr[2]
    position = (left_xm + right_xm) / 2 - center_xm
    if position >= 0:
        result.append("Vehicle is %.2fm to the left" % position)
    else:
        result.append("Vehicle is %.2fm to the right" % -position)
    result_string = '\n'.join(result)
#     print(result_string)
    return result_string, left_curverad, right_curverad

## 7. Draw curve on original image

In [ ]:
def draw_curve_on_image(image, undist, warped, Minv, left_fitx, right_fitx, ploty):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

## Pipeline

In [ ]:
def process_image(image):
    undist = cal_undistort(image, objpoints, imgpoints)
    color_binary, combined_binary = color_and_gradient_threshold(undist, s_thresh=(90, 255), sx_thresh=(20, 100))
    warpped_img, Minv = warp(combined_binary)

#     plt.figure()
#     plt.imshow(image)
#     plt.title("Original", fontsize=30)
    
#     plt.figure()
#     plt.imshow(undist)
#     plt.title("Undist", fontsize=30)

    plt.figure()
    plt.imshow(color_binary[:, :, 1], cmap="gray")
    plt.title("xGradient Binary", fontsize=30)
    
    plt.figure()
    plt.imshow(color_binary[:, :, 2], cmap="gray")
    plt.title("Color Binary", fontsize=30)
    
    plt.figure()
    plt.imshow(combined_binary, cmap="gray")
    plt.title("Binary", fontsize=30)

#     plt.figure()
#     plt.imshow(warpped_img, cmap="gray")
#     plt.title("Warpped", fontsize=30)
    
    out_img, leftx, lefty, rightx, righty = hist_and_slide_window(warpped_img)
    left_fit, right_fit, left_fitx, right_fitx, ploty = fit_curve(warpped_img, leftx, lefty, rightx, righty)
    
#     plot_curve(out_img, leftx, lefty, rightx, righty, left_fitx, right_fitx, ploty)
    result_string, left_curverad, right_curverad = cal_curverad(left_fit, right_fit, leftx, lefty, rightx, righty, ploty)
#     print(left_fitx)
#     print(right_fitx)
#     print(ploty)
    result = draw_curve_on_image(image, undist, warpped_img, Minv, left_fitx, right_fitx, ploty)
    
    plt.figure()
    plt.imshow(result)
    plt.annotate(result_string, xy=(0.05, 0.80), xycoords='axes fraction')
    return result

## Process images for testing purpose

In [ ]:
images = glob.glob('./test_images/test2.jpg')
for image_path in images:
    image = mpimg.imread(image_path)
#     print(image)
    process_image(image)

## Draw curve lane area on video

In [ ]:
def process_clip(clip1):
    cnt = 0
    new_frames = []
    left_smooth = []
    right_smooth = []
    smooth_num = 8
    skip_cnt = 0

    for image in clip1.iter_frames(fps=25,dtype='uint8',progress_bar=True):

        undist = cal_undistort(image, objpoints, imgpoints)
        color_binary, combined_binary = color_and_gradient_threshold(undist, s_thresh=(90, 255))
        warpped_img, Minv = warp(combined_binary)
        out_img, leftx, lefty, rightx, righty = hist_and_slide_window(warpped_img)
        left_fit, right_fit, left_fitx, right_fitx, ploty = fit_curve(warpped_img, leftx, lefty, rightx, righty)
        result_string, left_curverad, right_curverad = cal_curverad(left_fit, right_fit, leftx, lefty, rightx, righty, ploty)
        
        if cnt < smooth_num:
            left_smooth.append(left_fitx)
            right_smooth.append(right_fitx)
            cnt += 1
            continue
        
        # sanity check, if current lane position shifted too much(20%), skip current one 
        left_std = np.mean(left_smooth)
        right_std = np.mean(right_smooth)
        if (abs(np.mean(left_fitx)-left_std) / left_std < 0.2) and (abs(np.mean(right_fitx)-right_std) / right_std < 0.2):
            left_smooth.append(left_fitx)
            right_smooth.append(right_fitx)
        else:
            left_smooth.append(left_smooth[-1])
            right_smooth.append(right_smooth[-1])
            skip_cnt += 1

        left_mean = np.mean(left_smooth, axis=0)
        right_mean = np.mean(right_smooth, axis=0)

        result = draw_curve_on_image(image, undist, warpped_img, Minv, left_mean, right_mean, ploty)
        new_frames.append(result)

        left_smooth.pop(0)
        right_smooth.pop(0)
    print("%d frames are skipped." % skip_cnt)
    return new_frames

In [ ]:
white_output = './project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip('./project_video.mp4')
# white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
# %time white_clip.write_videofile(white_output, audio=False)
fps = 25

In [ ]:
new_frames = process_clip(clip1)
new_clip = ImageSequenceClip(new_frames, fps=fps)
new_clip.write_videofile('./project_video_output2.mp4') 

In [ ]:
# test code
clip2 = VideoFileClip('./project_video.mp4').subclip(20,27)
new_frames2 = process_clip(clip2)
new_clip = ImageSequenceClip(new_frames2, fps=fps)
new_clip.write_videofile('./project_video_output_sub.mp4') 

In [ ]:
# test code
clip3 = VideoFileClip('./project_video.mp4').subclip(25,25.5)
for image in clip3.iter_frames(fps=20,dtype='uint8',progress_bar=False):
    process_image(image)